### Clean Item Sale Data

#### Load Modules

In [43]:
import sys
import pandas as pd
sys.path.append("C:/Users/ping/MyDrive/py_files/python/py379/")
from myUtils import pickle_load, pickle_dump

#### Set Constants

In [44]:
pd.set_option('max_colwidth', 12, 'display.max_columns', 18, 'display.width', 1200, 'display.max_rows', 100)
INPUT_DIR = 'C:/Users/ping/OneDrive/Documents/jenn_bb_sales'
path_pickle_dump = f'{INPUT_DIR}/'

#### Load Item Sale and Weather Dataframe

In [45]:
df = pickle_load(path_pickle_dump, 'df_item_sale_n_weather_raw')
rows_org, cols_org = df.shape[0], df.shape[1]
_rows_dropped, _cols_dropped = (rows_org - df.shape[0]), (cols_org - df.shape[1])
print(f'df.shape: {df.shape}, rows dropped from original: {_rows_dropped}, columns dropped from original: {_cols_dropped}')

df.shape: (18366, 43), rows dropped from original: 0, columns dropped from original: 0


#### Drop Wholesale Customers

In [46]:
df = df[df['Customer Name'] != 'Canyon Coffee']
# print(f'df.shape: {df.shape}')
_rows_dropped, _cols_dropped = (rows_org - df.shape[0]), (cols_org - df.shape[1])
print(f'df.shape: {df.shape}, rows dropped from original: {_rows_dropped}, columns dropped from original: {_cols_dropped}')

df.shape: (18364, 43), rows dropped from original: 2, columns dropped from original: 0


#### In columns with string dtype, strip leading and trailing spaces, converts first character of each word to uppercase

In [47]:
# https://stackoverflow.com/questions/65756553/check-if-entire-pandas-object-column-is-a-string
for column in df.columns:
  # infer column cell type: 'string' or 'floating'
  col_type = pd.api.types.infer_dtype(df[column])
  if col_type == 'string':
    # https://www.datasciencemadesimple.com/strip-space-column-pandas-dataframe-leading-trailing-2/
    df[column] = df[column].str.strip()
    df[column] = df[column].str.title()

#### Combine Same Items

In [48]:
df.Item = df.Item.replace(to_replace=['Double Chocolate Espresso Cookie', '2X Choc'], value='Double Chocolate Cookie')
df.Item = df.Item.replace(to_replace=['Choc Croissant', 'Choc Crx'], value='Chocolate Croissant')
df.Item = df.Item.replace(to_replace=['Crx'], value='Croissant')
df.Item = df.Item.replace(to_replace=['Ccc'], value='Chocolate Chip Cookie')
df.Item = df.Item.replace(to_replace=['Cinn Knot'], value='Cinnamon Knot')
# df.Item = df.Item.replace(to_replace=['Bluerberry Muffin', 'Bb Muf'], value='Blueberry Muffin')
df.Item = df.Item.replace(to_replace=['Bluerberry Muffin', 'Bb Muf', 'Blueberry Muffin'], value='Muffin')
_rows_dropped, _cols_dropped = (rows_org - df.shape[0]), (cols_org - df.shape[1])
print(f'df.shape: {df.shape}, rows dropped from original: {_rows_dropped}, columns dropped from original: {_cols_dropped}')

df.shape: (18364, 43), rows dropped from original: 2, columns dropped from original: 0


#### Drop transactions with: refunds, custom-amount, and negative quantities

In [49]:
df = df[df['Event Type'] != 'Refund']
df = df[df['Item'] != 'Custom Amount']
df = df[df['Qty'] >= 0]
df = df.sort_values(['Date', 'Time'])
_rows_dropped, _cols_dropped = (rows_org - df.shape[0]), (cols_org - df.shape[1])
print(f'df.shape: {df.shape}, rows dropped from original: {_rows_dropped}, columns dropped from original: {_cols_dropped}')

df.shape: (18007, 43), rows dropped from original: 359, columns dropped from original: 0


#### Group and count items in the Item column

In [50]:
s = df.Item
grouped = s.groupby(s)
_dict = {}
for key in grouped.groups.keys():
  _count = grouped.get_group(key).count()
  _dict[key] = _count

_dict = dict(sorted(_dict.items(), reverse = True, key=lambda item: item[1]))
for k, v in _dict.items():
    print(f'{k:<45}{v:10,.0f}')

Danish                                            3,412
Chocolate Croissant                               2,433
Croissant                                         2,015
Chocolate Chip Cookie                             1,731
Cinnamon Knot                                     1,437
Double Chocolate Cookie                           1,207
Twice Baked                                         698
Coffee Cake                                         606
Jarred Pastry                                       575
Muffin                                              535
Furikake Crx                                        484
Oatmeal Cookie                                      440
Confetti Cookie                                     429
Ling Mut                                            346
Jam Bar                                             205
Molasses                                            202
Box                                                 111
Cake Slice                                      

#### Drop columns that are not needed

In [51]:
cols_df = df.columns.tolist()
cols_grSale_keep = \
  ['Date', 'Time', 'Category', 'Item', 'Qty', 'Gross Sales', 'Discounts', 'Net Sales', 'Tax', 'Transaction ID', 'Event Type', 'Dining Option', 'Customer Name']
cols_weather_keep = ['NAME', 'AWND', 'PRCP', 'TAVG', 'TMAX', 'TMIN', 'WT01', 'WT02', 'WT08', 'DOW']
cols_keep = cols_grSale_keep + cols_weather_keep
diff = set(cols_df) - set(cols_keep)
cols_drop = [item for item in cols_df if item in diff]  # retaining column order of set difference
df.drop(cols_drop, axis=1, inplace=True)
_rows_dropped, _cols_dropped = (rows_org - df.shape[0]), (cols_org - df.shape[1])
print(f'df.shape: {df.shape}, rows dropped from original: {_rows_dropped}, columns dropped from original: {_cols_dropped}')
print(f'columns dropped: {list(diff)}')
print(f'columns kept: {list(df.columns)}')
# print(f'df.columns: {df.columns}')
# print(f'df.shape: {df.shape}')

df.shape: (18007, 23), rows dropped from original: 359, columns dropped from original: 20
columns dropped: ['Payment ID', 'Itemization Type', 'STATION', 'SKU', 'Unit', 'WSF2', 'Location', 'WDF5', 'Time Zone', 'Count', 'PGTM', 'WDF2', 'Device Name', 'WSF5', 'Customer ID', 'Customer Reference ID', 'Details', 'Notes', 'Price Point Name', 'Modifiers Applied']
columns kept: ['Date', 'Time', 'Category', 'Item', 'Qty', 'Gross Sales', 'Discounts', 'Net Sales', 'Tax', 'Transaction ID', 'Event Type', 'Dining Option', 'Customer Name', 'NAME', 'AWND', 'PRCP', 'TAVG', 'TMAX', 'TMIN', 'WT01', 'WT02', 'WT08', 'DOW']


#### Calculate avg. temp.

In [52]:
df.TAVG = (df.TMAX + df.TMIN) / 2

#### Replace Day-Of-Week from str to integer 

In [53]:
df['DOW'] = df['DOW'].replace(to_replace=['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat'], value=[0, 1, 2, 3, 4, 5, 6])

#### Strip leading $, convert object to float and pickle df

In [54]:
_cols = ['Gross Sales', 'Discounts', 'Net Sales', 'Tax']
for _col in _cols:
  df[_col] = df[_col].str.split('$').str[-1]  # strip leading $
  df[_col] = pd.to_numeric(df[_col])  # convert from object to float
pickle_dump(df, path_pickle_dump, 'df_item_sale_n_weather_cleaned')

#### Re-load saved data

In [55]:
df = pickle_load(path_pickle_dump, 'df_item_sale_n_weather_cleaned')
print(f'df.shape: {df.shape}')
print(f'df.info(): {df.info()}')

df.shape: (18007, 23)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 18007 entries, 0 to 18363
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            18007 non-null  object 
 1   Time            18007 non-null  object 
 2   Category        18007 non-null  object 
 3   Item            18007 non-null  object 
 4   Qty             18007 non-null  float64
 5   Gross Sales     18007 non-null  float64
 6   Discounts       18007 non-null  float64
 7   Net Sales       18007 non-null  float64
 8   Tax             18007 non-null  float64
 9   Transaction ID  18007 non-null  object 
 10  Event Type      18007 non-null  object 
 11  Dining Option   17702 non-null  object 
 12  Customer Name   11290 non-null  object 
 13  NAME            18007 non-null  object 
 14  AWND            17856 non-null  float64
 15  PRCP            18007 non-null  float64
 16  TAVG            18007 non-null  float64
 17  TMAX     

#### List first and last date and time of the data set

In [56]:
ls = df.Date.to_list()
print(f'First and last Date of data set: {ls[0]:>10} to {ls[-1]:>10} {type(ls[-1])}')
ls = df.Time.to_list()
print(f'First and last Time of data set: {ls[0]:>10} to {ls[-1]:>10} {type(ls[-1])}')

First and last Date of data set: 2021-07-08 to 2022-09-04 <class 'str'>
First and last Time of data set:   17:31:07 to   11:39:13 <class 'str'>


#### Group and count items in each column

In [57]:
# https://datagy.io/pandas-groupby/
for column in df.columns[2::]: # skip columns: Date, Time
  s = df[column]
  grouped = s.groupby(s)
  print(f'Column: {column}')
  print('='*20)
  _dict = {}
  for key in grouped.groups.keys():
    _count = grouped.get_group(key).count()
    _dict[key] = _count
  
  # sort items and counts in the column in reverse order 
  _dict = dict(sorted(_dict.items(), reverse = True, key=lambda item: item[1]))
  for k, v in _dict.items():
      print(f'{k:<45}{v:10,.0f}')
  print('='*20, '\n')

Column: Category
Food                                             17,359
Beverage                                            195
Merch                                               169
Rpg                                                 147
None                                                 80
Coffee & Tea                                         57

Column: Item
Danish                                            3,412
Chocolate Croissant                               2,433
Croissant                                         2,015
Chocolate Chip Cookie                             1,731
Cinnamon Knot                                     1,437
Double Chocolate Cookie                           1,207
Twice Baked                                         698
Coffee Cake                                         606
Jarred Pastry                                       575
Muffin                                              535
Furikake Crx                                        484
Oatmeal Cookie   

##### Weather Data Definition  
STATION = Weather station ID  
NAME = Weather station location  
Date = Date  
AWND = Average daily wind speed (meters per second or miles per hour as per user preference)  
PGTM = Peak gust time (hours and minutes, i.e., HHMM)  
PRCP = Precipitation (mm or inches as per user preference, inches to hundredths on Daily Form pdf file)  
TMAX = Maximum temperature, F  
TMIN = Minimum temperature, F  
WDF2 = Direction of fastest 2-minute wind (degrees)  
WDF5 = Direction of fastest 5-second wind (degrees)  
WSF2 = Fastest 2-minute wind speed (miles per hour or meters per second as per user preference)  
WSF5 = Fastest 5-second wind speed (miles per hour or meters per second as per user preference)  
WT01 = Fog, ice fog, or freezing fog (may include heavy fog)  
WT02 = Heavy fog or heaving freezing fog (not always distinguished from fog)  
WT08 = Smoke or haze

In [58]:
# # how many rows have one or more valid inputs in columns: WT01, WT02, WT08'
# _df = df[['WT01', 'WT02', 'WT08']]
# _idx = _df.index[_df.notnull().any(axis=1)]
# _df_notnull = _df.iloc[_idx]
# print(f'{_df_notnull.shape[0]} rows have one or more valid inputs in columns: WT01, WT02, WT08')
# print(f'_df_notnull.shape: {_df_notnull.shape}')